In [35]:
# Make project root importable and (re)load config cleanly
from pathlib import Path
import sys, importlib

NB_DIR = Path.cwd()
PROJECT_ROOT = NB_DIR.parent if NB_DIR.name == "notebooks" else NB_DIR
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import config
importlib.reload(config)  # ensure we see the latest edits

from config import ROOT, RAW_SHAPES, PROJ_SHAPES, TRACTS_RAW, CSAS_RAW, TRACTS_FP, CSAS_FP
print("CONFIG ROOT:", ROOT)


CONFIG ROOT: /Users/warrenjones/Dev/igs-analysis-baltimore


In [36]:
# make project root importable
from pathlib import Path
import sys

NB_DIR = Path.cwd()
PROJECT_ROOT = NB_DIR.parent if NB_DIR.name == "notebooks" else NB_DIR
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from config import ROOT, RAW_SHAPES, PROJ_SHAPES, TRACTS_RAW, CSAS_RAW, TRACTS_FP, CSAS_FP

print("CONFIG ROOT:", ROOT)


CONFIG ROOT: /Users/warrenjones/Dev/igs-analysis-baltimore


In [37]:
from config import ROOT, RAW_SHAPES, PROJ_SHAPES, TRACTS_RAW, CSAS_RAW, TRACTS_FP, CSAS_FP


In [21]:
import geopandas as gpd
from pathlib import Path

# --- define paths explicitly so we don’t rely on earlier cells ---
ROOT = Path("..") if (Path.cwd().name == "notebooks") else Path(".")
RAW_SHAPES = ROOT / "data_raw" / "shapes"

# raw TIGER and CSA reference files
TRACTS_RAW = RAW_SHAPES / "2020_Census_Tracts_(Census_TIGER).geojson"
CSAS_RAW   = RAW_SHAPES / "Community_Statistical_Areas_(CSAs)__Reference_Boundaries.geojson"

# --- load files ---
print("Loading:")
print("  ", TRACTS_RAW)
print("  ", CSAS_RAW)

metro = gpd.read_file(TRACTS_RAW)
csas  = gpd.read_file(CSAS_RAW)

print(f"Loaded tracts: {len(metro):,}")
print(f"Loaded CSAs:   {len(csas):,}")
metro.head(2)



Loading:
   ../data_raw/shapes/2020_Census_Tracts_(Census_TIGER).geojson
   ../data_raw/shapes/Community_Statistical_Areas_(CSAs)__Reference_Boundaries.geojson
Loaded tracts: 717
Loaded CSAs:   56


,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_Length,Shape_Area,geometry
0,1,24,003,731103,24003731103,7311.03,Census Tract,G5020,S,8562084,0,+39.0343178,-076.4946333,51684.935897,9.215257e+07,"POLYGON ((-76.52241 39.05171, -76.52232 39.051..."
1,2,24,003,707001,24003707001,7070.01,Census Tract,G5020,S,27326340,9371758,+38.8202066,-076.5292506,108449.152983,3.949741e+08,"POLYGON ((-76.58707 38.82953, -76.58707 38.829..."


In [6]:
import geopandas as gpd
from pathlib import Path

ROOT        = Path("..")                                # notebook is in /notebooks
RAW_SHAPES  = ROOT / "data_raw" / "shapes"
PROJ_SHAPES = ROOT / "shapes"
PROJ_SHAPES.mkdir(parents=True, exist_ok=True)

TRACTS_RAW = RAW_SHAPES / "2020_Census_Tracts_(Census_TIGER).geojson"
CSAS_RAW   = RAW_SHAPES / "Community_Statistical_Areas_(CSAs)__Reference_Boundaries.geojson"

print("Expecting files:")
print("  ", TRACTS_RAW)
print("  ", CSAS_RAW)

assert TRACTS_RAW.exists(), f"Missing: {TRACTS_RAW}"
assert CSAS_RAW.exists(),   f"Missing: {CSAS_RAW}"

metro = gpd.read_file(TRACTS_RAW)
csas  = gpd.read_file(CSAS_RAW)

print("\nTracts columns:", list(metro.columns))
print("CSAs columns   :", list(csas.columns))
metro.head(2)



Expecting files:
   ../data_raw/shapes/2020_Census_Tracts_(Census_TIGER).geojson
   ../data_raw/shapes/Community_Statistical_Areas_(CSAs)__Reference_Boundaries.geojson

Tracts columns: ['OBJECTID', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'GEOID20', 'NAME20', 'NAMELSAD20', 'MTFCC20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'Shape_Length', 'Shape_Area', 'geometry']
CSAs columns   : ['FID', 'Community', 'Neigh', 'Link', 'CSA2020', 'geometry']


,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_Length,Shape_Area,geometry
0,1,24,003,731103,24003731103,7311.03,Census Tract,G5020,S,8562084,0,+39.0343178,-076.4946333,51684.935897,9.215257e+07,"POLYGON ((-76.52241 39.05171, -76.52232 39.051..."
1,2,24,003,707001,24003707001,7070.01,Census Tract,G5020,S,27326340,9371758,+38.8202066,-076.5292506,108449.152983,3.949741e+08,"POLYGON ((-76.58707 38.82953, -76.58707 38.829..."


In [14]:
# --- Imports & paths ---
from pathlib import Path
import pandas as pd
import geopandas as gpd

# If you have pyogrio, GeoPandas reads much faster; otherwise it's fine.
# import pyogrio  # optional

# Project root (adjust if your notebook lives under /notebooks)
ROOT = Path.cwd()
DATA_RAW = ROOT / "data_raw"
SHAPES = DATA_RAW / "shapes"
OUT = ROOT / "data_intermediate"
OUT.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)


ROOT: /Users/warrenjones/Dev/igs-analysis-baltimore/notebooks


In [15]:
def load_points_any(fp):
    """
    Load point data from GeoJSON/GeoPackage/Shapefile and ensure a valid CRS.
    Returns a GeoDataFrame in EPSG:4326 by default.
    """
    gdf = gpd.read_file(fp)
    if gdf.empty:
        raise ValueError(f"No features found in {fp}")
    if gdf.crs is None:
        # If source CRS is missing, set it if you actually know it; defaulting to 4326 is common for GeoJSON.
        gdf.set_crs(4326, inplace=True)
    # Normalize to WGS84 for consistency
    gdf = gdf.to_crs(4326)
    return gdf


def load_polygons_any(fp, target_epsg=3857):
    """
    Load polygon data (e.g., Census tracts). Reprojects to a metric CRS for spatial ops.
    EPSG:3857 is fine for counts; use a local CRS if you need area-accurate calcs.
    """
    gdf = gpd.read_file(fp)
    if gdf.empty:
        raise ValueError(f"No features found in {fp}")
    if gdf.crs is None:
        gdf.set_crs(4326, inplace=True)
    return gdf.to_crs(target_epsg)


def to_metric(gdf, target_epsg=3857):
    if gdf.crs is None:
        gdf = gdf.set_crs(4326)
    return gdf.to_crs(target_epsg)


def points_to_tract_counts(points_gdf, tracts_gdf, tract_id_col="GEOID", count_col="points_count"):
    """
    Spatially join points to tracts and return a tract-level count GeoDataFrame.
    points_gdf: GeoDataFrame with Point geometries
    tracts_gdf: GeoDataFrame with tract polygons (metric CRS recommended)
    """
    # Reproject points to match tracts for spatial join
    pts = points_gdf
    if pts.crs is None:
        pts = pts.set_crs(4326)
    pts = pts.to_crs(tracts_gdf.crs)

    joined = gpd.sjoin(pts, tracts_gdf[[tract_id_col, "geometry"]], how="left", predicate="within")
    counts = (joined
              .groupby(tract_id_col, dropna=False)
              .size()
              .rename(count_col)
              .reset_index())

    out = tracts_gdf.merge(counts, on=tract_id_col, how="left")
    out[count_col] = out[count_col].fillna(0).astype(int)
    return out


def save_feat(gdf, name_no_ext, out_dir=OUT):
    """
    Save both Parquet (attributes) and GeoParquet (geometry).
    Creates <name>.parquet (GeoParquet if available) under data_intermediate by default.
    """
    out_fp = out_dir / f"{name_no_ext}.parquet"
    # If geopandas >=0.13 with pyarrow supports GeoParquet:
    try:
        gdf.to_parquet(out_fp)
    except Exception:
        # Fallback: drop geometry to plain Parquet and also write a GeoJSON for the map
        tbl_fp = out_dir / f"{name_no_ext}_table.parquet"
        gj_fp = out_dir / f"{name_no_ext}.geojson"
        gdf.drop(columns="geometry").to_parquet(tbl_fp)
        gdf.to_file(gj_fp, driver="GeoJSON")
        return str(tbl_fp), str(gj_fp)
    return str(out_fp)


In [23]:
from pathlib import Path
import geopandas as gpd

# ---- Ensure paths exist even after a restart ----
ROOT = Path("..") if Path.cwd().name == "notebooks" else Path(".")
PROJ_SHAPES = ROOT / "shapes"
PROJ_SHAPES.mkdir(parents=True, exist_ok=True)

# ---- Ensure 'metro' exists (load raw tracts if needed) ----
try:
    metro  # defined earlier?
except NameError:
    RAW_SHAPES = ROOT / "data_raw" / "shapes"
    TRACTS_RAW = RAW_SHAPES / "2020_Census_Tracts_(Census_TIGER).geojson"
    metro = gpd.read_file(TRACTS_RAW)

# ---- Normalize column names, preserve geometry ----
geom_col = metro.geometry.name  # usually 'geometry'
metro_up = metro.rename(columns=str.upper)
if "GEOMETRY" in metro_up.columns and "geometry" not in metro_up.columns:
    metro_up = metro_up.rename(columns={"GEOMETRY": "geometry"})

# ---- Choose TIGER fields with fallbacks ----
STATE = "STATEFP"   if "STATEFP"   in metro_up.columns else "STATEFP20"
COUNT = "COUNTYFP"  if "COUNTYFP"  in metro_up.columns else "COUNTYFP20"
GEOID = "GEOID"     if "GEOID"     in metro_up.columns else (
        "GEOID20" if "GEOID20" in metro_up.columns else "GEOID10"
)

# ---- Filter Maryland (24) -> Baltimore City (510) ----
bal_city = metro_up.query(f"{STATE} == '24' and {COUNT} == '510'").copy()
bal_city["tract_id"] = bal_city[GEOID].astype(str)

# ---- Keep only id + geometry, preserve CRS ----
bal_city = gpd.GeoDataFrame(
    bal_city[["tract_id", "geometry"]],
    geometry="geometry",
    crs=metro_up.crs
)

# ---- Save standardized project copies ----
TRACTS_FP = PROJ_SHAPES / "baltimore_tracts_2020.geojson"
CSAS_FP   = PROJ_SHAPES / "baltimore_csa.geojson"  # you’ll overwrite this later from your CSA cell
bal_city.to_file(TRACTS_FP, driver="GeoJSON")

print(f"Saved {len(bal_city)} tracts  → {TRACTS_FP}")
print(f"(CSA path reserved)          → {CSAS_FP}")



Saved 199 tracts  → ../shapes/baltimore_tracts_2020.geojson
(CSA path reserved)          → ../shapes/baltimore_csa.geojson


In [8]:
import geopandas as gpd
import pandas as pd

tracts = gpd.read_file(TRACTS_FP)  # has 'tract_id'
csas   = gpd.read_file(CSAS_FP)

# pick a CSA id column that exists
if "CSA2010" in csas.columns:
    csa_id_col = "CSA2010"
elif "CSA_NAME" in csas.columns:
    csa_id_col = "CSA_NAME"
elif "NAME" in csas.columns:
    csa_id_col = "NAME"
else:
    # last resort: first non-geometry column
    csa_id_col = next(c for c in csas.columns if c != csas.geometry.name)

csas = csas.rename(columns={csa_id_col: "csa_id"})[["csa_id", "geometry"]]

# project to a metric CRS for reliable area math
proj = tracts.estimate_utm_crs()
tracts_p = tracts.to_crs(proj)
csas_p   = csas.to_crs(proj)

# intersection + weights
cross = gpd.overlay(tracts_p, csas_p, how="intersection")
cross["area_overlap"] = cross.geometry.area

tract_area = cross.groupby("tract_id")["area_overlap"].sum().rename("tract_area")
cross = cross.merge(tract_area, on="tract_id", how="left")
cross["weight_area"] = (cross["area_overlap"] / cross["tract_area"]).clip(0, 1)

crosswalk = cross[["tract_id", "csa_id", "weight_area"]].sort_values(["tract_id","csa_id"])

# QA: per-tract weights should sum ~ 1.0
qa = crosswalk.groupby("tract_id")["weight_area"].sum().round(6).describe()
qa



count    199.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: weight_area, dtype: float64

In [9]:
from pathlib import Path

OUT = ROOT / "data_clean" / "spatial_crosswalks"
OUT.mkdir(parents=True, exist_ok=True)
CW_FP = OUT / "csa_to_tract.parquet"
crosswalk.to_parquet(CW_FP, index=False)
print("Saved crosswalk →", CW_FP, "| rows:", len(crosswalk))


Saved crosswalk → ../data_clean/spatial_crosswalks/csa_to_tract.parquet | rows: 595


In [10]:
import pandas as pd
from pathlib import Path

CW_FP = ROOT / "data_clean/spatial_crosswalks" / "csa_to_tract.parquet"
CLEAN_COMM = ROOT / "data_clean" / "community"
CLEAN_COMM.mkdir(parents=True, exist_ok=True)

cw = pd.read_parquet(CW_FP)

def _pick_csa_key(df):
    for k in ("CSA2010","CSA_NAME","NAME","CSA_ID"):
        if k in df.columns:
            return k
    # last resort: first non-numeric column that isn't geometry-ish
    for c in df.columns:
        if df[c].dtype == object and c.lower() not in ("shape_area","shape_length","geometry"):
            return c
    raise ValueError("Could not find a CSA key column.")

def _ensure_numeric(df, col):
    # coerce with commas/percent signs handled
    return pd.to_numeric(df[col].astype(str).str.replace(",","").str.replace("%",""), errors="coerce")

def save_parquet(df, filename):
    out = CLEAN_COMM / filename
    df.to_parquet(out, index=False)
    print("saved →", out, "| rows:", len(df))
    return out


In [15]:
# Cell 5 (replace): Downscale a CSA %/rate to tracts, robust
import pandas as pd
import re

COMM_DIR = ROOT / "data_raw" / "community"

# 1) pick the exact CSV you want (or let pattern find it)
# Pattern looks for "High_School" & "Some_College"
pattern = re.compile(r"High[_ ]School.*Some[_ ]College", re.I)
candidates = [p.name for p in COMM_DIR.glob("*.csv")]
match = next((f for f in candidates if pattern.search(f)), None)
if not match:
    raise FileNotFoundError("Could not find the HS/Some College file. Pick one from:\n" + "\n".join(candidates))

INPUT_CSV = COMM_DIR / match
print("Using:", INPUT_CSV)

# 2) load + clean columns
csa = pd.read_csv(INPUT_CSV)
csa.columns = [c.strip() for c in csa.columns]

# 3) choose CSA key and a real percent/rate column (exclude IDs/geometry)
def pick_csa_key(df):
    for k in ("CSA2010","CSA_NAME","NAME","CSA_ID"):
        if k in df.columns:
            return k
    # fallback: first text column that isn't geometry-ish
    for c in df.columns:
        if df[c].dtype == object and c.lower() not in ("shape_area","shape_length","geometry","objectid"):
            return c
    raise ValueError("No CSA key found.")

def pick_value_col(df):
    # prefer semantic names
    for pat in (r"^percent$", r"percent", r"\bpct\b", r"rate", r"per_?1000"):
        cols = [c for c in df.columns if re.search(pat, c, re.I)]
        cols = [c for c in cols if c.lower() not in ("shape_area","shape_length","objectid")]
        if cols:
            return cols[0]
    # fallback: numeric column with most non-null values (excluding id-ish)
    num_cols = [c for c in df.columns
                if c.lower() not in ("objectid","shape_area","shape_length") and
                   pd.to_numeric(df[c], errors="coerce").notna().sum() > 0]
    if not num_cols:
        raise ValueError("No numeric value column found.")
    return num_cols[0]

CSA_KEY  = pick_csa_key(csa)
VALUE_COL= pick_value_col(csa)
print("CSA key:", CSA_KEY, "| value column:", VALUE_COL)

# 4) harmonize types and clean numbers
csa = csa.rename(columns={CSA_KEY: "csa_id"})
csa["csa_id"] = csa["csa_id"].astype(str).str.strip()
cw["csa_id"]  = cw["csa_id"].astype(str).str.strip()

csa[VALUE_COL] = pd.to_numeric(
    csa[VALUE_COL].astype(str).str.replace(",", "", regex=False).str.replace("%","", regex=False),
    errors="coerce"
)

# 5) merge + area-weight downscale
m = csa.merge(cw, on="csa_id", how="inner")
m["weighted_value"] = m[VALUE_COL] * m["weight_area"]

tract_rate = (m.groupby("tract_id", as_index=False)["weighted_value"]
                .sum()
                .rename(columns={"weighted_value": "pct_hs_or_more_areaweighted"}))

save_parquet(tract_rate, "tract_pct_hs_or_more.parquet")
tract_rate.head()



Using: ../data_raw/community/Percent_Population_(25_Years_and_over)_With_High_School_Diploma_and_Some_College_or_Associates_Degree.csv
CSA key: CSA2010 | value column: hsdipl11
saved → ../data_clean/community/tract_pct_hs_or_more.parquet | rows: 0


,tract_id,pct_hs_or_more_areaweighted


In [16]:
# Cell 6 (replace): Downscale a per-1,000 CSA metric to tracts (area-weighted)
import pandas as pd
import re

COMM_DIR = ROOT / "data_raw" / "community"
pattern = re.compile(r"Banks?_and_Bank_Branches.*1,?000", re.I)
candidates = [p.name for p in COMM_DIR.glob("*.csv")]
match = next((f for f in candidates if pattern.search(f)), None)
if not match:
    raise FileNotFoundError("Could not find the Banks per 1,000 file. Pick one from:\n" + "\n".join(candidates))

INPUT_CSV = COMM_DIR / match
print("Using:", INPUT_CSV)

csa = pd.read_csv(INPUT_CSV)
csa.columns = [c.strip() for c in csa.columns]

def pick_csa_key(df):
    for k in ("CSA2010","CSA_NAME","NAME","CSA_ID"):
        if k in df.columns:
            return k
    for c in df.columns:
        if df[c].dtype == object and c.lower() not in ("shape_area","shape_length","geometry","objectid"):
            return c
    raise ValueError("No CSA key found.")

def pick_value_col(df):
    # look for per-1000-ish column names
    for pat in (r"per.?1,?000", r"per_?1000", r"rate"):
        cols = [c for c in df.columns if re.search(pat, c, re.I)]
        cols = [c for c in cols if c.lower() not in ("shape_area","shape_length","objectid")]
        if cols:
            return cols[0]
    # fallback to most numeric column excluding ids
    num_cols = [c for c in df.columns
                if c.lower() not in ("objectid","shape_area","shape_length") and
                   pd.to_numeric(df[c], errors="coerce").notna().sum() > 0]
    if not num_cols:
        raise ValueError("No numeric value column found.")
    return num_cols[0]

CSA_KEY  = pick_csa_key(csa)
VALUE_COL= pick_value_col(csa)
print("CSA key:", CSA_KEY, "| value column:", VALUE_COL)

csa = csa.rename(columns={CSA_KEY: "csa_id"})
csa["csa_id"] = csa["csa_id"].astype(str).str.strip()
cw["csa_id"]  = cw["csa_id"].astype(str).str.strip()

csa[VALUE_COL] = pd.to_numeric(
    csa[VALUE_COL].astype(str).str.replace(",", "", regex=False),
    errors="coerce"
)

m = csa.merge(cw, on="csa_id", how="inner")
m["weighted_value"] = m[VALUE_COL] * m["weight_area"]

tract_banks = (m.groupby("tract_id", as_index=False)["weighted_value"]
                 .sum()
                 .rename(columns={"weighted_value": "banks_per_1000_areaweighted"}))

save_parquet(tract_banks, "tract_banks_per_1000.parquet")
tract_banks.head()


Using: ../data_raw/community/Number_of_Banks_and_Bank_Branches_per_1,000_Residents_-_Community_Statistical_Area.csv
CSA key: CSA2010 | value column: banks11
saved → ../data_clean/community/tract_banks_per_1000.parquet | rows: 0


,tract_id,banks_per_1000_areaweighted


In [17]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

TRACTS_FP = ROOT / "shapes" / "baltimore_tracts_2020.geojson"
tracts = gpd.read_file(TRACTS_FP)
tracts = tracts[["tract_id","geometry"]]

def load_points_any(path: Path, lon_col=None, lat_col=None, crs_epsg=4326):
    """
    Load a point layer from CSV/GeoJSON/Shapefile.
    If CSV, you must pass lon_col/lat_col. Returns a GeoDataFrame in tract CRS.
    """
    path = Path(path)
    if path.suffix.lower() == ".csv":
        assert lon_col and lat_col, "For CSV points, provide lon_col and lat_col."
        df = pd.read_csv(path)
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[lon_col], df[lat_col], crs=f"EPSG:{crs_epsg}")
        )
    else:
        gdf = gpd.read_file(path)
        if gdf.crs is None:
            gdf.set_crs(epsg=crs_epsg, inplace=True)
    # project to match tracts
    return gdf.to_crs(tracts.crs)

def points_to_tract_counts(points_gdf, id_col="tract_id", how="within"):
    """Spatial join → counts per tract."""
    joined = gpd.sjoin(points_gdf, tracts, how="left", predicate=how)
    counts = (joined.groupby(id_col).size()
                    .reset_index(name="count")
                    .rename(columns={id_col:"tract_id"}))
    return counts

def nearest_distance_to_feature(points_gdf, k=1):
    """
    For each tract polygon centroid, compute distance to nearest point (in meters).
    Returns DataFrame tract_id, dist_nearest_m.
    """
    # work in a metric CRS
    proj = tracts.estimate_utm_crs()
    tr_p = tracts.to_crs(proj).copy()
    pts_p = points_gdf.to_crs(proj).copy()

    # build spatial index
    sindex = pts_p.sindex

    def nearest_dist(geom):
        cand_idx = list(sindex.nearest(geom.bounds, num_results=k))
        return float(geom.distance(pts_p.iloc[cand_idx[0]].geometry))

    tr_p["dist_nearest_m"] = tr_p.geometry.centroid.apply(nearest_dist)
    return tr_p[["tract_id","dist_nearest_m"]].copy()

def save_feat(df: pd.DataFrame, name: str):
    out = CLEAN_COMM / name
    df.to_parquet(out, index=False)
    print("saved →", out, "| rows:", len(df))
    return out


In [20]:
# ============================================================
# Cell 8 — utilities for converting point layers into tract-level features
# ============================================================

import geopandas as gpd
import pandas as pd
from pathlib import Path

TRACTS_FP = ROOT / "shapes" / "baltimore_tracts_2020.geojson"
tracts = gpd.read_file(TRACTS_FP)[["tract_id", "geometry"]]

def load_points_any(path: Path, lon_col=None, lat_col=None, crs_epsg=4326):
    """
    Load a point layer from CSV/GeoJSON/Shapefile.
    • Auto-detects longitude/latitude field names if not given.
    • Returns a GeoDataFrame projected to match the tracts layer.
    """
    path = Path(path)

    if path.suffix.lower() == ".csv":
        df = pd.read_csv(path)

        # --- Auto-detect lon/lat fields ---
        def pick(cols, cands):
            for c in cols:
                cl = c.lower().replace(" ", "").replace("-", "").replace("_", "")
                if cl in cands:
                    return c
            return None

        cols = list(df.columns)

        if not lon_col or not lat_col:
            lon_candidates = {
                "longitude","long","lon","x","pointx","xcoord","xcoordinate",
                "locationlongitude","lng"
            }
            lat_candidates = {
                "latitude","lat","y","pointy","ycoord","ycoordinate",
                "locationlatitude"
            }
            lon_col = lon_col or pick(cols, lon_candidates)
            lat_col = lat_col or pick(cols, lat_candidates)

        if lon_col is None or lat_col is None:
            raise KeyError(
                f"Couldn't find lon/lat columns. Columns are: {cols}. "
                f"Pass lon_col=..., lat_col=... explicitly."
            )

        # --- build geometry ---
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                pd.to_numeric(df[lon_col], errors="coerce"),
                pd.to_numeric(df[lat_col], errors="coerce"),
                crs=f"EPSG:{crs_epsg}"
            )
        ).dropna(subset=["geometry"])

    else:
        # --- handle GeoJSON, Shapefile, etc. ---
        gdf = gpd.read_file(path)
        if gdf.crs is None:
            gdf.set_crs(epsg=crs_epsg, inplace=True)

    return gdf.to_crs(tracts.crs)

def points_to_tract_counts(points_gdf, id_col="tract_id", how="within"):
    """Spatial join points to tracts and count points per tract."""
    joined = gpd.sjoin(points_gdf, tracts, how="left", predicate=how)
    counts = (joined.groupby(id_col)
                    .size()
                    .reset_index(name="count")
                    .rename(columns={id_col: "tract_id"}))
    return counts

def nearest_distance_to_feature(points_gdf, k=1):
    """
    For each tract centroid, compute distance to nearest point (in meters).
    Returns DataFrame [tract_id, dist_nearest_m].
    """
    proj = tracts.estimate_utm_crs()
    tr_p = tracts.to_crs(proj).copy()
    pts_p = points_gdf.to_crs(proj).copy()
    sindex = pts_p.sindex

    def nearest_dist(geom):
        cand_idx = list(sindex.nearest(geom.bounds, num_results=k))
        return float(geom.distance(pts_p.iloc[cand_idx[0]].geometry))

    tr_p["dist_nearest_m"] = tr_p.geometry.centroid.apply(nearest_dist)
    return tr_p[["tract_id", "dist_nearest_m"]].copy()

def save_feat(df: pd.DataFrame, name: str):
    out = CLEAN_COMM / name
    df.to_parquet(out, index=False)
    print("saved →", out, "| rows:", len(df))
    return out


In [21]:
# quick check — see what columns the grocery CSV actually has
grocery_fp = ROOT / "data_raw" / "community" / "Grocery_Stores.csv"  # adjust if the filename differs
cols = pd.read_csv(grocery_fp, nrows=2).columns.tolist()
print("Grocery file columns:", cols)


Grocery file columns: ['X', 'Y', 'OBJECTID', 'latitude', 'longitude', 'address', 'storename', 'in_oz', 'label', 'renovationincentiveelig', 'status', 'oz_sc', 'congressionaldistrict_md', 'congressionaldistrict_usno', 'md_ld', 'GlobalID']


In [22]:
grocery = load_points_any(grocery_fp)  # auto-detect will work
gr_counts = points_to_tract_counts(grocery)
save_feat(gr_counts, "tract_groceries_count.parquet")


saved → ../data_clean/community/tract_groceries_count.parquet | rows: 0


PosixPath('../data_clean/community/tract_groceries_count.parquet')

In [10]:
# === Helpers & setup (safe to rerun) ===
import geopandas as gpd
import pandas as pd
from pathlib import Path

ROOT        = Path("..")  # notebook lives in /notebooks
CLEAN_COMM  = ROOT / "data_clean" / "community"
CLEAN_COMM.mkdir(parents=True, exist_ok=True)

TRACTS_FP = ROOT / "shapes" / "baltimore_tracts_2020.geojson"
tracts = gpd.read_file(TRACTS_FP)[["tract_id", "geometry"]]

def load_points_any(path: Path, lon_col=None, lat_col=None, crs_epsg=4326):
    """
    Load a point layer from CSV/GeoJSON/Shapefile.
    Auto-detects lon/lat names if not provided. Returns GeoDataFrame in tract CRS.
    """
    path = Path(path)
    if path.suffix.lower() == ".csv":
        df = pd.read_csv(path)

        def pick(cols, cands):
            for c in cols:
                cl = c.lower().replace(" ", "").replace("-", "").replace("_", "")
                if cl in cands: return c
            return None

        cols = list(df.columns)
        if not lon_col or not lat_col:
            lon_col = lon_col or pick(cols, {
                "longitude","long","lon","x","pointx","xcoord","xcoordinate","locationlongitude","lng"
            })
            lat_col = lat_col or pick(cols, {
                "latitude","lat","y","pointy","ycoord","ycoordinate","locationlatitude"
            })
        if lon_col is None or lat_col is None:
            raise KeyError(f"Couldn't find lon/lat. Columns: {cols}. Pass lon_col=..., lat_col=...")

        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                pd.to_numeric(df[lon_col], errors="coerce"),
                pd.to_numeric(df[lat_col], errors="coerce"),
                crs=f"EPSG:{crs_epsg}"
            )
        ).dropna(subset=["geometry"])
    else:
        gdf = gpd.read_file(path)
        if gdf.crs is None:
            gdf.set_crs(epsg=crs_epsg, inplace=True)

    return gdf.to_crs(tracts.crs)

def points_to_tract_counts(points_gdf, id_col="tract_id", how="within"):
    joined = gpd.sjoin(points_gdf, tracts, how="left", predicate=how)
    return (joined.groupby(id_col).size()
                  .reset_index(name="count")
                  .rename(columns={id_col: "tract_id"}))

def nearest_distance_to_feature(points_gdf, k=1):
    proj = tracts.estimate_utm_crs()
    tr_p = tracts.to_crs(proj).copy()
    pts_p = points_gdf.to_crs(proj).copy()
    sindex = pts_p.sindex
    def nearest_dist(geom):
        idx = list(sindex.nearest(geom.bounds, num_results=k))
        return float(geom.distance(pts_p.iloc[idx[0]].geometry))
    tr_p["dist_nearest_m"] = tr_p.geometry.centroid.apply(nearest_dist)
    return tr_p[["tract_id","dist_nearest_m"]]

def save_feat(df: pd.DataFrame, name: str):
    out = CLEAN_COMM / name
    df.to_parquet(out, index=False)
    print("saved →", out, "| rows:", len(df))
    return out


In [26]:
# --- Load your school points ---
schools_fp = ROOT / "data_raw" / "shapes" / "Baltimore_City_Schools.geojson"
schools = load_points_any(schools_fp)  # GeoJSON already has geometry

# --- Load the standardized Baltimore tracts from your shapes folder ---
tracts_fp = ROOT / "shapes" / "baltimore_tracts_2020.geojson"
tracts = load_polygons_any(tracts_fp)  # function reprojects for spatial ops

# --- Compute counts ---
sch_counts = points_to_tract_counts(schools, tracts, tract_id_col="tract_id", count_col="schools_count")

# --- Save results ---
save_feat(sch_counts, "tract_schools_count.parquet")



'/Users/warrenjones/Dev/igs-analysis-baltimore/notebooks/data_intermediate/tract_schools_count.parquet.parquet'

In [27]:
# 1) CRS sanity
print("schools:", schools.crs, "| tracts:", tracts.crs)

# 2) join coverage — % of points assigned to a tract
pts_total = len(schools)
pts_joined = sch_counts["schools_count"].sum()
print("points total:", pts_total, " | joined:", pts_joined, " | coverage:", round(100*pts_joined/pts_total, 2), "%")

# 3) tract_id integrity
assert sch_counts["tract_id"].notna().all()
assert sch_counts["tract_id"].is_unique

# 4) counts are nonnegative integers
assert (sch_counts["schools_count"] >= 0).all()
assert (sch_counts["schools_count"].dropna().astype(int) == sch_counts["schools_count"]).all()


schools: EPSG:4326 | tracts: EPSG:3857
points total: 155  | joined: 155  | coverage: 100.0 %


In [28]:
# config.py
from pathlib import Path
ROOT = Path("..") if Path.cwd().name == "notebooks" else Path(".")
RAW_SHAPES  = ROOT / "data_raw" / "shapes"
PROJ_SHAPES = ROOT / "shapes"
TRACTS_RAW  = RAW_SHAPES / "2020_Census_Tracts_(Census_TIGER).geojson"
CSAS_RAW    = RAW_SHAPES / "Community_Statistical_Areas_(CSAs)__Reference_Boundaries.geojson"
TRACTS_FP   = PROJ_SHAPES / "baltimore_tracts_2020.geojson"
CSAS_FP     = PROJ_SHAPES / "baltimore_csa.geojson"
